In [1]:
import re
import pandas as pd
import os
import csv

In [2]:
input_dir = './most_popular_sneakers/'
file_list = [f for f in os.listdir(input_dir) if not f.startswith('.')]

In [3]:
file_list

['jordan_4_2019.txt',
 'nike_all_2017.txt',
 'jordan_4_2018.txt',
 'adidas_nmd_2017.txt',
 'jordan_1_2017.txt',
 'adidas_yeezy_2018.txt',
 'adidas_yeezy_2019.txt',
 'jordan_6_2017.txt',
 'nike_other_2018.txt',
 'jordan_11_2018.txt',
 'adidas_ultraboost_2018.txt',
 'adidas_ultraboost_2019.txt',
 'jordan_11_2019.txt',
 'other_2017.txt',
 'jordan_11_2017.txt',
 'adidas_ultraboost_2017.txt',
 'other_2019.txt',
 'other_2018.txt',
 'jordan_6_2019.txt',
 'jordan_6_2018.txt',
 'nike_af_2017.txt',
 'nike_af_2018',
 'jordan_1_2019.txt',
 'jordan_1_2018.txt',
 'adidas_yeezy_2017.txt',
 'nike_all_2018.txt',
 'jordan_4_2017.txt',
 'nike_all_2019.txt',
 'adidas_nmd_2018.txt']

In [4]:
tmp = input_dir + file_list[-5]
with open(tmp,'r') as f:
        name = [line for line in f if 'name' in line]
with open(tmp,'r') as f:
    rawDates = [line for line in f if 'releaseDate' in line]
    replaceDates = [w.replace(':', '-') for w in rawDates]
    formattedDates = [w.replace('releaseDate', '') for w in replaceDates]
with open(tmp,'r') as f:
    brand = [line for line in f if '"brand" :' in line]
with open(tmp,'r') as f:
    model = [line for line in f if '"model" :' in line]
with open(tmp,'r') as f:
    sku = [line for line in f if '"sku" :' in line]
with open(tmp,'r') as f:
    color = [line for line in f if '"color" : ' in line]
with open(tmp,'r') as f:
    image_url = [line for line in f if '"image" : ' in line]
# with open(tmp,'r') as f:
#     url_dup = [line for line in f if '"url"' in line]
#     url = [k for k in url_dup if '#' not in k]
#url = [k for k in url_dup if '#' not in k]

In [5]:
def parse_stockx(file):
    with open(file,'r') as f:
        name = [line for line in f if 'name' in line]
    with open(file,'r') as f:
        rawDates = [line for line in f if 'releaseDate' in line]
        replaceDates = [w.replace(':', '-') for w in rawDates]
        formattedDates = [w.replace('releaseDate', '') for w in replaceDates]
    with open(file,'r') as f:
        brand = [line for line in f if 'brand' in line]
    with open(file,'r') as f:
        model = [line for line in f if 'model' in line]
    with open(file,'r') as f:
        sku = [line for line in f if 'sku' in line]
    with open(file,'r') as f:
        color = [line for line in f if '"color" : ' in line]
    with open(file,'r') as f:
        image_url = [line for line in f if '"image" : ' in line]
#     with open(file,'r') as f:
#         url_dup = [line for line in f if '"url" : ' in line]
#         url = [k for k in url_dup if '#' not in k]
    
    df = pd.DataFrame({'name':name,
            'releaseDate':formattedDates,
            'brand':brand,
            'model':model,
            'sku':sku,
            'color':color,
            'image':image_url})
#                        ,'url':url})
    return(df)

In [6]:
def format_data(df):
    df = df.applymap(lambda x: x.replace('"', ''))
    for i in df.columns:
        df[str(i)] = df[str(i)].str.split(':').str[-1].str.strip()
        df[str(i)] = df[str(i)].str.replace(',', '')
    
    # creat new column for url scraping
    front = 'https://stockx.com/api/products/'
    end = '/activity?state=480&currency=USD&limit=20000&page=1&sort=createdAt&order=DESC'
    add_https = 'https:'
    
    df['api_url'] = front + df['sku'] + end
    df['image'] = add_https + df['image']
#     df['url'] = add_https + df['url']
    df['releaseDate'] = df['releaseDate'].str[2:]
    return(df)

In [7]:
def OutputURLs(input_dir):
    file_list = [f for f in os.listdir(input_dir) if not f.startswith('.')]
    
    d = {'name':[],
     'releaseDate':[],
     'brand':[],
     'model':[],
     'sku':[],
     'color':[],
     'api_url':[]}

    df = pd.DataFrame(d)
    
    for file in file_list:
        file_str = input_dir + ''.join(file)
        data = parse_stockx(file_str)
        formatted_df = format_data(data)

        brand = file_str.split('/')[-1].split('.')[0]
        out_dir = './sneaker_urls/'
        out_name = brand + '_urls.csv'
        out_path = out_dir + out_name
        
        formatted_df.api_url.to_csv(out_path, index=False)
        
        print('URLs for ' + brand + ' have been saved!')

        df = pd.concat([df, formatted_df])
        print(brand + ' has been successfully appended')
        
    return(df)

In [8]:
input_dir = './most_popular_sneakers/'

In [9]:
df = OutputURLs(input_dir)

/Users/allenchang/PycharmProjects/insight/venv/lib/python3.7/site-packages/ipykernel_launcher.py:24: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
/Users/allenchang/PycharmProjects/insight/venv/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



URLs for jordan_4_2019 have been saved!
jordan_4_2019 has been successfully appended
URLs for nike_all_2017 have been saved!
nike_all_2017 has been successfully appended
URLs for jordan_4_2018 have been saved!
jordan_4_2018 has been successfully appended
URLs for adidas_nmd_2017 have been saved!
adidas_nmd_2017 has been successfully appended
URLs for jordan_1_2017 have been saved!
jordan_1_2017 has been successfully appended
URLs for adidas_yeezy_2018 have been saved!
adidas_yeezy_2018 has been successfully appended
URLs for adidas_yeezy_2019 have been saved!
adidas_yeezy_2019 has been successfully appended
URLs for jordan_6_2017 have been saved!
jordan_6_2017 has been successfully appended
URLs for nike_other_2018 have been saved!
nike_other_2018 has been successfully appended
URLs for jordan_11_2018 have been saved!
jordan_11_2018 has been successfully appended
URLs for adidas_ultraboost_2018 have been saved!
adidas_ultraboost_2018 has been successfully appended
URLs for adidas_ultra

In [11]:
unique_sku = df.groupby("sku").first()

In [12]:
unique_sku['sku'] = unique_sku.index

In [13]:
unique_sku.to_csv('sneaker_metadata.csv')

In [18]:
top_brands = ['Jordan', 'adidas', 'Nike']

ni_ad_jo = unique_sku[unique_sku['brand'].isin(top_brands)]

In [21]:
ni_ad_jo.to_csv('nike_adidas_jordan_metadata.csv')